In [1]:
import os
# 한글 지원
os.putenv('NLS_LANG', '.UTF8')

In [2]:
def ipchecker():    
    import urllib 
    import re
    
    pattern = re.compile(b'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')
    dyndns = urllib.request.urlopen('http://checkip.dyndns.org').read()
    
    wan = re.search(pattern, dyndns).group().decode('utf-8')
    
    return wan

In [3]:
# base db_connect param

local_ip = '200.1.220.251:1521/xe' # 원내에서 접속할 db_ip
outside_ip = '61.35.152.103:1521/xe' # 원밖에서 접속할 db_ip
localhost = 'localhost:1521/xe'

db_id = 'begopa' 
db_pwd = 'begopa1'

academy_ip = '61.35.152.66'
db_ip = local_ip if academy_ip == ipchecker() else outside_ip

db_ip = localhost # 집에서 본인의 개인 컴퓨터dptj  오라클에 접속할때는

In [4]:
import pickle
# load object ::
# :: file_nm
# :: columns and data_type
with open("./pickle/col2dict.pickle","rb") as fr:
    col2dict = pickle.load(fr)
with open("./pickle/table_nm_set.pickle","rb") as fr:
    table_nm_set = pickle.load(fr)

In [5]:
path = "./sg-data/"
desc_path = path + 'desc/'
col_path = path + 'col/'
origin_path = path + 'origin/'

file_list = os.listdir(origin_path)

In [6]:
# 자치구별 상권 변화지표 db 에서 한글 컬럼으로 가져오기
# oracle_to_df('자치구별 상권변화지표')
def load_to_df(kr_tbl_nm):
    for key, value in table_nm_set.items():
        if kr_tbl_nm == value:
            table_nm = key
            columns = [ arr[0] for arr in col2dict[key] ]
           
    import cx_Oracle
    import pandas as pd
    
    db = cx_Oracle.connect(db_id, db_pwd, db_ip) 
        
    sql = 'select * from ' + table_nm

    df = pd.read_sql(sql=sql, con = db) 
    df.columns = columns
    db.close
    
    return df

In [7]:
import re
import os

file_list = os.listdir(origin_path)
for file in file_list:
    kr_tbl_nm = re.findall('(\d\d).*\((.*)\)',file)
    print(kr_tbl_nm[0][0], kr_tbl_nm[0][1])

00 상권-영역
01 상권-추정유동인구
02 상권-집객시설
03 상권배후지-추정유동인구
04 상권배후지-집객시설
05 상권배후지-상주인구
06 상권-상주인구
07 상권배후지-아파트
08 상권배후지-점포
09 상권-상권변화지표
10 행정동별 상권변화지표
11 상권-추정매출
12 상권-직장인구
13 상권배후지-소득소비
14 상권배후지-직장인구
15 자치구별 상권변화지표
16 상권-아파트
18 상권-점포
19 상권배후지-추정매출


In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [23]:
def multi_columns_merging3(sg, sg_hl):
    a_df = load_to_df( sg )
    b_df = load_to_df( sg_hl )
    all_df = pd.merge(left=a_df, right=b_df, how='left', \
                  on=['기준_년_분기_코드','상권_코드'], sort=False)
#     del a_df, b_df
    return all_df

def multi_columns_merging4(a_df, b_df):
    all_df = pd.merge(left=a_df, right=b_df, how='left', \
                  on=['기준_년_분기_코드','상권_코드'], sort=False)
#     del a_df, b_df
    return all_df

def multi_columns_merging5(a_df, b_df):
    all_df = pd.merge(left=a_df, right=b_df, how='left', \
                  on=['기준_년_분기_코드','상권_코드'], sort=False)
#     del a_df, b_df
    return all_df.dropna(axis=0)

In [10]:
fac_df=multi_columns_merging3('상권-집객시설', '상권배후지-집객시설').fillna('0')

In [11]:
fac_df.head()
len(fac_df['상권_코드'].unique())

1462

In [12]:
apt=multi_columns_merging3('상권-아파트', '상권배후지-아파트').fillna('0')

In [13]:
# apt.columns.to_list()

In [14]:
apt_df=apt.drop(apt.filter(regex='아파트_면적|아파트_가격'),axis=1)

In [15]:
# apt_df[apt_df['상권_코드']=='1001010']
len(apt_df['상권_코드'].unique())

1322

In [16]:
def sg_svc_category_merge(df):
    grouped = df.groupby(['기준_년_분기_코드','상권_코드'])
    grouped_part1 = grouped.count() # 각 상권별 서비스 업종 코드수    
    grouped_part2 = grouped.sum() # 각 상권별 서비스 업종 분류수    
    join_grouped = grouped_part1.iloc[:,:1].join(grouped_part2, on=['기준_년_분기_코드', '상권_코드'], how='inner')
    return join_grouped

def multi_columns_merging_svc_cd(sg, sg_hl):
    jumpo_df = load_to_df(sg)
    hl_jumpo_df = load_to_df(sg_hl)
    
    jumpo_df = sg_svc_category_merge(jumpo_df)
    hl_jumpo_df = sg_svc_category_merge(hl_jumpo_df)
    
    hl_jumpo_df.rename(columns={'서비스_업종_코드':'hl_서비스_업종_코드'})
    
    all_df = pd.merge(left=jumpo_df, right=hl_jumpo_df, how='left', \
                  on=['기준_년_분기_코드','상권_코드'], sort=False)
    del jumpo_df, hl_jumpo_df
    
# jumpo_df.index.name
# 멀티 인덱스 제거
    
    return all_df.dropna(axis=0).reset_index(['상권_코드','기준_년_분기_코드'])

In [17]:
store=multi_columns_merging_svc_cd('상권-점포', '상권배후지-점포')

In [18]:
store.head()
len(store['상권_코드'].unique())

1010

In [20]:
h_total_a=multi_columns_merging4(fac_df, apt_df).fillna('0')

In [21]:
len(h_total_a['상권_코드'].unique())

1462

In [28]:
total=multi_columns_merging4(h_total_a, store).fillna('0')

In [29]:
len(total['상권_코드'].unique())

1462